# Car Detection

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/car_detection.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [1]:
 %pip install geoai-py

## Import libraries

In [2]:
import geoai

## Download sample data

We will download a sample image from Hugging Face Hub to use for car detection. You can find more high-resolution images from [OpenAerialMap](https://openaerialmap.org).

In [3]:
raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/cars_7cm.tif"
)

In [4]:
raster_path = geoai.download_file(raster_url)

cars_7cm.tif: 100%|██████████| 92.0M/92.0M [00:01<00:00, 95.4MB/s]


## Visualize the image

In [5]:
geoai.view_raster(raster_url)

## Initialize the model

In [6]:
detector = geoai.CarDetector()

Model path not specified, downloading from Hugging Face...


car_detection_usa.pth:   0%|          | 0.00/176M [00:00<?, ?B/s]

Model downloaded to: /root/.cache/huggingface/hub/models--giswqs--geoai/snapshots/75788bf6253e967a91b9576b1620949a3b939820/car_detection_usa.pth
Model loaded successfully


## Extract cars

Extract cars from the image using the model and save the output image.

In [7]:
mask_path = detector.generate_masks(
    raster_path=raster_path,
    output_path="cars_masks.tif",
    confidence_threshold=0.3,
    mask_threshold=0.5,
    overlap=0.25,
    chip_size=(400, 400),
)

Dataset initialized with 15 rows and 28 columns of chips
Image dimensions: 8351 x 4463 pixels
Chip size: 400 x 400 pixels
Overlap: 25.0% (stride_x=300, stride_y=300)
CRS: EPSG:3857
Processing raster with 105 batches


100%|██████████| 105/105 [00:44<00:00,  2.38it/s]


Masks with confidence values saved to cars_masks.tif


Convert the image masks to polygons and save the output GeoJSON file.

In [8]:
gdf = detector.vectorize_masks(
    masks_path="cars_masks.tif",
    output_path="cars.geojson",
    min_object_area=100,
    max_object_area=2000,
)

Processing masks from: cars_masks.tif
Found 1012 connected components
Using single-threaded processing, you can speed up processing by setting n_workers > 1


Processing components: 100%|██████████| 1012/1012 [01:31<00:00, 11.08it/s]


Saved 101 objects with confidence to cars.geojson
Total processing time: 92.29 seconds


## Add geometric properties

In [9]:
gdf = geoai.add_geometric_properties(gdf)

## Visualize initial results

In [10]:
geoai.view_vector_interactive(gdf, column="confidence", tiles=raster_url)

## Filter cars by area

In [11]:
gdf_filter = gdf[
    (gdf["area_m2"] > 8) & (gdf["area_m2"] < 60) & (gdf["minor_length_m"] > 1)
]

In [12]:
len(gdf_filter)

55

## Visualiza final results

In [13]:
geoai.view_vector_interactive(gdf_filter, column="confidence", tiles=raster_url)

In [14]:
geoai.view_vector_interactive(gdf_filter, tiles=raster_url)

![image](https://github.com/user-attachments/assets/a1e4c871-b152-466a-b902-7c00e62e5444)